In [1]:
PROJECT_ID = 'razekmh-angola'
# import libraries
import ee
import geemap
import json

In [2]:
# authenticate to Earth Engine
ee.Authenticate()
# initialize the project    
ee.Initialize(project=PROJECT_ID)

In [3]:
# make a map
Map = geemap.Map(center=[-12.0, 18.0], zoom=8)

In [4]:
# read the project geojson
with open('Kassanje bound_FeaturesToJSO.geojson') as f:
    kassanje_data = json.load(f)
# convert geojson to ee FeatureCollection
ee_kassanje_polygon = ee.Geometry.Polygon(kassanje_data['features'][0]['geometry']['coordinates'][0])
ee_kassanje_polygon_feature = ee.FeatureCollection(ee_kassanje_polygon)


In [5]:
# read the buffer project geojson
with open('KassanjBuff20ToJSO.geojson') as f:
    kassanje_buff = json.load(f)
# convert geojson to ee FeatureCollection
ee_kassanje_buffer= ee.Geometry.Polygon(kassanje_buff['features'][0]['geometry']['coordinates'][0])
ee_kassanje_buffer_feature = ee.FeatureCollection(ee_kassanje_buffer)


In [6]:
landsat_8_image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
    .filterBounds(ee_kassanje_polygon_feature) \
        .filterDate('2020-01-01', '2024-08-01') \
            .filterMetadata('CLOUD_COVER', 'less_than', 10)

# Applies scaling factors. ref # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#colab-python
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

landsat_8_image_collection = landsat_8_image_collection.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B7', 'SR_B5', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

In [10]:
# make region for export

ee_kassanje_polygon.bounds().getInfo()['coordinates'][0]

def get_bbox_coords(ee_polygon):
    coordinate_list = ee_polygon.bounds().getInfo()['coordinates'][0][:-1]
    east_west_list = [coord[0] for coord in coordinate_list]
    north_south_list = [coord[1] for coord in coordinate_list]
    west = min(east_west_list)
    east = max(east_west_list)
    south = min(north_south_list)
    north = max(north_south_list)
    return west, south, east, north
west, south, east, north = get_bbox_coords(ee_kassanje_polygon)
region = ee.Geometry.BBox(west, south, east, north)

In [11]:
# make image for export
mosaic_landsat8 = landsat_8_image_collection.mosaic()
export_image = mosaic_landsat8.select('SR_B7', 'SR_B5', 'SR_B3')

# make crs for export
crs = export_image.projection().crs().getInfo()


In [16]:
# export the image
task = ee.batch.Export.image.toDrive(
    image=export_image,
    description='image_export',
    folder='ee_demos',
    region=region,
    scale=30,
    crs=crs,
    maxPixels=1e13
)
task.start()

In [22]:
ee.data.getTaskList()

[{'state': 'RUNNING',
  'description': 'image_export',
  'priority': 100,
  'creation_timestamp_ms': 1729619254281,
  'update_timestamp_ms': 1729619386042,
  'start_timestamp_ms': 1729619259317,
  'task_type': 'EXPORT_IMAGE',
  'attempt': 1,
  'id': 'Y7UZZIT6RWZOVDGCSFXTCYQW',
  'name': 'projects/razekmh-angola/operations/Y7UZZIT6RWZOVDGCSFXTCYQW'},
 {'state': 'FAILED',
  'description': 'image_export',
  'priority': 100,
  'creation_timestamp_ms': 1729619133513,
  'update_timestamp_ms': 1729619138940,
  'start_timestamp_ms': 1729619137634,
  'task_type': 'EXPORT_IMAGE',
  'attempt': 1,
  'error_message': 'Export too large: specified 915740804 pixels (max: 100000000). Specify higher maxPixels value if you intend to export a large area.',
  'id': 'D7KRSSB3SRK7H4ZDXJUUCNMK',
  'name': 'projects/razekmh-angola/operations/D7KRSSB3SRK7H4ZDXJUUCNMK'}]

In [7]:
# add outline of Angola to the map

countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
angola_outline = countries.filter(ee.Filter.eq('ADM0_NAME', 'Angola'))
Map.addLayer(angola_outline, {}, 'Angola')

In [9]:
# display the map
Map.addLayer(landsat_8_image_collection, visualization, 'Landsat 8')
Map.addLayer(ee_kassanje_buffer_feature, {}, 'Kassanje buffer')

In [10]:
# Mosaic and display the map
mosaic_landsat8 = landsat_8_image_collection.mosaic()
Map.addLayer(mosaic_landsat8, visualization, 'mosaic.Landsat_8')
Map.addLayer(ee_kassanje_buffer_feature, {}, 'Kassanje buffer')
Map

Map(center=[-12.0, 18.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [11]:
#clip image 
mosaic_landsat8.clip(ee_kassanje_polygon_feature)
Map.addLayer(mosaic_landsat8.clip(ee_kassanje_polygon_feature), visualization, 'clip.Landsat_8')
Map.addLayer(mosaic_landsat8.clip(ee_kassanje_buffer_feature), visualization, 'clip.bufferLandsat_8')
Map

Map(center=[-12.0, 18.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [14]:
ee_kassanje_buffer_feature

In [ ]:
ee_kassanje_buffer_feature.geometry().getInfo()

In [21]:
#import 753RGBcomposite landsat8 image, region
image_landsat8 = ee.Image(mosaic_landsat8.clip(ee_kassanje_polygon_feature)).select(
    ["SR_B7", "SR_B5", "SR_B3"]
)
ee_kassanje_buffer= ee.Geometry.Polygon(kassanje_buff['features'][0]['geometry']['coordinates'][0])
ee_kassanje_buffer_feature = ee.FeatureCollection(ee_kassanje_buffer)
style = {"color": "ffff00ff", "fillColor": "00000000"}
#export 753RGBcomposite landsat8 image
projection = image_landsat8.select(0).projection().getInfo()

In [30]:
mosaic_visualization = mosaic_landsat8.select(["SR_B7", "SR_B5", "SR_B3"])

In [32]:
task = ee.batch.Export.image.toDrive(
    image=mosaic_visualization,
    description='imageToDriveExample_transform',
    crs=projection['crs'],
    crsTransform=projection['transform'],
    region=ee_kassanje_buffer_feature.geometry(),
    fileFormat='GeoTIFF',
    formatOptions={'cloudOptimized': True},
)
task.start()

In [ ]:
geemap.ee_export_image(image_landsat8, filename="landsat8.tif", scale=90, region=ee_kassanje_buffer)

In [20]:
#convertion of javascript pca_cod to python
js_snippet= """
// Script showing how to do Principal Component Analysis on images
var composite = ee.Image("users/ujavalgandhi/e2e/arkavathy_2019_composite");
var boundary = ee.FeatureCollection("users/ujavalgandhi/e2e/arkavathy_boundary");

print('Input Composite', composite);
Map.centerObject(composite)
Map.addLayer(composite, {bands: ['B4', 'B3', 'B2'], min: 0, max: 3000, gamma: 1.2}, 'RGB');

// Define the geometry and scale parameters
var geometry = boundary.geometry();
var scale = 20;

// Run the PCA function
var pca = PCA(composite)

// Extract the properties of the pca image
var variance = pca.toDictionary()
print('Variance of Principal Components', variance)

// As you see from the printed results, ~97% of the variance
// from the original image is captured in the first 3 principal components
// We select those and discard others
var pca = PCA(composite).select(['pc1', 'pc2', 'pc3'])
print('First 3 PCA Bands', pca);

// PCA computation is expensive and can time out when displaying on the map
// Export the results and import them back
Export.image.toAsset({
  image: pca,
  description: 'Principal_Components_Image',
  assetId: 'users/ujavalgandhi/e2e/arkavathy_pca',
  region: geometry,
  scale: scale,
  maxPixels: 1e10})
// Once the export finishes, import the asset and display
var pcaImported = ee.Image('users/ujavalgandhi/e2e/arkavathy_pca')
var pcaVisParams = {bands: ['pc1', 'pc2', 'pc3'], min: -2, max: 2};

Map.addLayer(pcaImported, pcaVisParams, 'Principal Components');


//************************************************************************** 
// Function to calculate Principal Components
// Code adapted from https://developers.google.com/earth-engine/guides/arrays_eigen_analysis
//************************************************************************** 
function PCA(maskedImage){
  var image = maskedImage.unmask()
  var scale = scale;
  var region = geometry;
  var bandNames = image.bandNames();
  // Mean center the data to enable a faster covariance reducer
  // and an SD stretch of the principal components.
  var meanDict = image.reduceRegion({
    reducer: ee.Reducer.mean(),
    geometry: region,
    scale: scale,
    maxPixels: 1e13,
    tileScale: 16
  });
  var means = ee.Image.constant(meanDict.values(bandNames));
  var centered = image.subtract(means);
  // This helper function returns a list of new band names.
  var getNewBandNames = function(prefix) {
    var seq = ee.List.sequence(1, bandNames.length());
    return seq.map(function(b) {
      return ee.String(prefix).cat(ee.Number(b).int());
    });
  };
  // This function accepts mean centered imagery, a scale and
  // a region in which to perform the analysis.  It returns the
  // Principal Components (PC) in the region as a new image.
  var getPrincipalComponents = function(centered, scale, region) {
    // Collapse the bands of the image into a 1D array per pixel.
    var arrays = centered.toArray();
    
    // Compute the covariance of the bands within the region.
    var covar = arrays.reduceRegion({
      reducer: ee.Reducer.centeredCovariance(),
      geometry: region,
      scale: scale,
      maxPixels: 1e13,
      tileScale: 16
    });

    // Get the 'array' covariance result and cast to an array.
    // This represents the band-to-band covariance within the region.
    var covarArray = ee.Array(covar.get('array'));

    // Perform an eigen analysis and slice apart the values and vectors.
    var eigens = covarArray.eigen();

    // This is a P-length vector of Eigenvalues.
    var eigenValues = eigens.slice(1, 0, 1);
    
    // Compute Percentage Variance of each component
    // This will allow us to decide how many components capture
    // most of the variance in the input
    var eigenValuesList = eigenValues.toList().flatten()
    var total = eigenValuesList.reduce(ee.Reducer.sum())

    var percentageVariance = eigenValuesList.map(function(item) {
      var component = eigenValuesList.indexOf(item).add(1).format('%02d')
      var variance = ee.Number(item).divide(total).multiply(100).format('%.2f')
      return ee.List([component, variance])
    })
    // Create a dictionary that will be used to set properties on final image
    var varianceDict = ee.Dictionary(percentageVariance.flatten())
    // This is a PxP matrix with eigenvectors in rows.
    var eigenVectors = eigens.slice(1, 1);
    // Convert the array image to 2D arrays for matrix computations.
    var arrayImage = arrays.toArray(1);

    // Left multiply the image array by the matrix of eigenvectors.
    var principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage);

    // Turn the square roots of the Eigenvalues into a P-band image.
    // Call abs() to turn negative eigenvalues to positive before
    // taking the square root
    var sdImage = ee.Image(eigenValues.abs().sqrt())
      .arrayProject([0]).arrayFlatten([getNewBandNames('sd')]);

    // Turn the PCs into a P-band image, normalized by SD.
    return principalComponents
      // Throw out an an unneeded dimension, [[]] -> [].
      .arrayProject([0])
      // Make the one band array image a multi-band image, [] -> image.
      .arrayFlatten([getNewBandNames('pc')])
      // Normalize the PCs by their SDs.
      .divide(sdImage)
      .set(varianceDict);
  };
  var pcImage = getPrincipalComponents(centered, scale, region);
  return pcImage.mask(maskedImage.mask());
}"""

In [21]:
geemap.js_snippet_to_py(js_snippet,add_new_cell=True,show_map=True)

An error occurred: list assignment index out of range. The closing curly bracket could not be found in Line 73: var getPrincipalComponents = function(centered, scale, region) {
. Please reformat the function definition and make sure that both the opening and closing curly brackets appear on the same line as the function keyword. 


In [ ]:
#CONVERSION code without addition and modification
import ee

# Script showing how to do Principal Component Analysis on images
composite = ee.Image("users/ujavalgandhi/e2e/arkavathy_2019_composite")
boundary = ee.FeatureCollection("users/ujavalgandhi/e2e/arkavathy_boundary")

print('Input Composite', composite.getInfo())
m.centerObject(composite)
m.addLayer(composite, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 3000, 'gamma': 1.2}, 'RGB')

# Define the geometry and scale parameters
geometry = boundary.geometry()
scale = 20

# Run the PCA function
pca = PCA(composite)

# Extract the properties of the pca image
variance = pca.toDictionary()
print('Variance of Principal Components', variance.getInfo())

# As you see from the printed results, ~97% of the variance
# from the original image is captured in the first 3 principal components
# We select those and discard others
pca = PCA(composite).select(['pc1', 'pc2', 'pc3'])
print('First 3 PCA Bands', pca.getInfo())

# PCA computation is expensive and can time out when displaying on the map
# Export the results and import them back
geemap.ee_export_image_to_asset(
image=pca,
description='Principal_Components_Image',
assetId='users/ujavalgandhi/e2e/arkavathy_pca',
region=geometry,
scale=scale,
maxPixels=1e10})
# Once the export finishes, import the asset and display
pcaImported = ee.Image('users/ujavalgandhi/e2e/arkavathy_pca')
pcaVisParams = {bands=['pc1', 'pc2', 'pc3'], min=-2, max=2}

m.addLayer(pcaImported, pcaVisParams, 'Principal Components')

#**************************************************************************
# Function to calculate Principal Components
# Code adapted from https =#developers.google.com/earth-engine/guides/arrays_eigen_analysis
#**************************************************************************
def PCA(maskedImage) =
    image = maskedImage.unmask()
    scale = scale
    region = geometry
    bandNames = image.bandNames()
    # Mean center the data to enable a faster covariance reducer
    # and an SD stretch of the principal components.
    meanDict = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=scale,
    maxPixels=1e13,
    tileScale=16
    )
    means = ee.Image.constant(meanDict.values(bandNames))
    centered = image.subtract(means)
    # This helper function returns a list of new band names.

    def func_loy(prefix) =
        seq = ee.List.sequence(1, bandNames.length())

        def func_yls(b) =
                return ee.String(prefix).cat(ee.Number(b).int())

        return seq.map(func_yls)

    getNewBandNames = func_loy

    def func_xak(centered, scale, region) =
        # Collapse the bands of the image into a 1D array per pixel.
        arrays = centered.toArray()

        # Compute the covariance of the bands within the region.
        covar = arrays.reduceRegion(
            reducer=ee.Reducer.centeredCovariance(),
            geometry=region,
            scale=scale,
            maxPixels=1e13,
            tileScale=16
            )

        # Get the 'array' covariance result and cast to an array.
        # This represents the band-to-band covariance within the region.
        covarArray = ee.Array(covar.get('array'))

        # Perform an eigen analysis and slice apart the values and vectors.
        eigens = covarArray.eigen()

        # This is a P-length vector of Eigenvalues.
        eigenValues = eigens.slice(1, 0, 1)

        # Compute Percentage Variance of each component
        # This will allow us to decide how many components capture
        # most of the variance in the input
        eigenValuesList = eigenValues.toList().flatten()
        total = eigenValuesList.reduce(ee.Reducer.sum())

        def func_vbw(item) =
                component = eigenValuesList.indexOf(item).add(1).format('%02d')
                variance = ee.Number(item).divide(total).multiply(100).format('%.2f')
                return ee.List([component, variance])

        percentageVariance = eigenValuesList.map(func_vbw)

        # Create a dictionary that will be used to set properties on final image
        varianceDict = ee.Dictionary(percentageVariance.flatten())
        # This is a PxP matrix with eigenvectors in rows.
        eigenVectors = eigens.slice(1, 1)
        # Convert the array image to 2D arrays for matrix computations.
        arrayImage = arrays.toArray(1)

        # Left multiply the image array by the matrix of eigenvectors.
        principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

        # Turn the square roots of the Eigenvalues into a P-band image.
        # Call abs() to turn negative eigenvalues to positive before
        # taking the square root
        sdImage = ee.Image(eigenValues.abs().sqrt()) \
        .arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

        # Turn the PCs into a P-band image, normalized by SD.
        return principalComponents \
        .arrayProject([0]) \
        .arrayFlatten([getNewBandNames('pc')]) \
        .divide(sdImage)

m

In [24]:
#CONVERSION PCA_code with addition and modification

# Script showing how to do Principal Component Analysis on images
composite = ee.Image("mosaic_landsat8.clip(ee_kassanje_polygon_feature)_composite")
boundary = ee.FeatureCollection(ee_kassanje_buffer)

print('Input Composite', composite.getInfo())
Map.centerObject(composite)
Map.addLayer(composite, {'bands': ['SR_B4',  'SR_B3',  'SR_B2'], 'min': 0, 'max': 3000, 'gamma': 1.2}, 'RGB')

# Define the geometry and scale parameters
geometry = boundary.geometry()
scale = 30

# Run the PCA function
pca = PCA(composite)

# Extract the properties of the pca image
variance = pca.toDictionary()
print('Variance of Principal Components', variance.getInfo())

# As you see from the printed results, ~97% of the variance
# from the original image is captured in the first 3 principal components
# We select those and discard others
pca = PCA(composite).select(['pc1', 'pc2', 'pc3'])
print('First 3 PCA Bands', pca.getInfo())

# PCA computation is expensive and can time out when displaying on the map
# Export the results and import them back6h
geemap.ee_export_image_to_asset({
    image=pca,
    description='Principal_Components_Image',
    assetId='mosaic_landsat8.clip(ee_kassanje_polygon_feature)_pca',
    region=geometry,
    scale=scale,
    maxPixels=1e10})
# Once the export finishes, import the asset and display
pcaImported = ee.Image('mosaic_landsat8.clip(ee_kassanje_polygon_feature)_pca')
pcaVisParams = {bands=['pc1', 'pc2', 'pc3'], min=-2, max=2}

Map.addLayer(pcaImported, pcaVisParams, 'Principal Components')

#**************************************************************************
# Function to calculate Principal Components
# Code adapted from https =#developers.google.com/earth-engine/guides/arrays_eigen_analysis
#**************************************************************************
def PCA(maskedImage) =
    image = maskedImage.unmask()
    scale = scale
    region = geometry
    bandNames = image.bandNames()
    # Mean center the data to enable a faster covariance reducer
    # and an SD stretch of the principal components.
    meanDict = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=scale,
    maxPixels=1e13,
    tileScale=16
    )
    means = ee.Image.constant(meanDict.values(bandNames))
    centered = image.subtract(means)
    # This helper function returns a list of new band names.

    def func_loy(prefix) =
        seq = ee.List.sequence(1, bandNames.length())

        def func_yls(b) =
                return ee.String(prefix).cat(ee.Number(b).int())

        return seq.map(func_yls)

    getNewBandNames = func_loy

    def func_xak(centered, scale, region) =
        # Collapse the bands of the image into a 1D array per pixel.
        arrays = centered.toArray()

        # Compute the covariance of the bands within the region.
        covar = arrays.reduceRegion(
            reducer=ee.Reducer.centeredCovariance(),
            geometry=region,
            scale=scale,
            maxPixels=1e13,
            tileScale=16
            )

        # Get the 'array' covariance result and cast to an array.
        # This represents the band-to-band covariance within the region.
        covarArray = ee.Array(covar.get('array'))

        # Perform an eigen analysis and slice apart the values and vectors.
        eigens = covarArray.eigen()

        # This is a P-length vector of Eigenvalues.
        eigenValues = eigens.slice(1, 0, 1)

        # Compute Percentage Variance of each component
        # This will allow us to decide how many components capture
        # most of the variance in the input
        eigenValuesList = eigenValues.toList().flatten()
        total = eigenValuesList.reduce(ee.Reducer.sum())

        def func_vbw(item) =
                component = eigenValuesList.indexOf(item).add(1).format('%02d')
                variance = ee.Number(item).divide(total).multiply(100).format('%.2f')
                return ee.List([component, variance])

        percentageVariance = eigenValuesList.map(func_vbw)

        # Create a dictionary that will be used to set properties on final image
        varianceDict = ee.Dictionary(percentageVariance.flatten())
        # This is a PxP matrix with eigenvectors in rows.
        eigenVectors = eigens.slice(1, 1)
        # Convert the array image to 2D arrays for matrix computations.
        arrayImage = arrays.toArray(1)

        # Left multiply the image array by the matrix of eigenvectors.
        principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

        # Turn the square roots of the Eigenvalues into a P-band image.
        # Call abs() to turn negative eigenvalues to positive before
        # taking the square root
        sdImage = ee.Image(eigenValues.abs().sqrt()) \
        .arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

        # Turn the PCs into a P-band image, normalized by SD.
        return principalComponents \
        .arrayProject([0]) \
        .arrayFlatten([getNewBandNames('pc')]) \
        .divide(sdImage)

Map

SyntaxError: invalid syntax (2071811192.py, line 31)

In [25]:
#conversion of spectral signature code to python
js_snippet= """
var gcps = ee.FeatureCollection("users/ujavalgandhi/e2e/bangalore_gcps");
var composite = ee.Image('users/ujavalgandhi/e2e/bangalore_composite');

// Overlay the point on the image to get bands data.
var training = composite.sampleRegions({
  collection: gcps, 
  properties: ['landcover'], 
  scale: 10
});


// We will create a chart of spectral signature for all classes

// We have multiple GCPs for each class
// Use a grouped reducer to calculate the average reflectance
// for each band for each class

// We have 12 bands so need to repeat the reducer 12 times
// We also need to group the results by class
// So we find the index of the landcover property and use it
// to group the results
var bands = composite.bandNames()
var numBands = bands.length()
var bandsWithClass = bands.add('landcover')
var classIndex = bandsWithClass.indexOf('landcover')

// Use .combine() to get a reducer capable of 
// computing multiple stats on the input
var combinedReducer = ee.Reducer.mean().combine({
  reducer2: ee.Reducer.stdDev(),
  sharedInputs: true})

// Use .repeat() to get a reducer for each band
// We then use .group() to get stats by class
var repeatedReducer = combinedReducer.repeat(numBands).group(classIndex)

var gcpStats = training.reduceColumns({
    selectors: bands.add('landcover'),
    reducer: repeatedReducer,
})

// Result is a dictionary, we do some post-processing to
// extract the results
var groups = ee.List(gcpStats.get('groups'))

var classNames = ee.List(['urban', 'bare', 'water', 'vegetation'])

var fc = ee.FeatureCollection(groups.map(function(item) {
  // Extract the means
  var values = ee.Dictionary(item).get('mean')
  var groupNumber = ee.Dictionary(item).get('group')
  var properties = ee.Dictionary.fromLists(bands, values)
  var withClass = properties.set('class', classNames.get(groupNumber))
  return ee.Feature(null, withClass)
}))

// Chart spectral signatures of training data
var options = {
  title: 'Average Spectral Signatures',
  hAxis: {title: 'Bands'},
  vAxis: {title: 'Reflectance'},
  lineWidth: 1,
  pointSize: 4,
  series: {
    0: {color: 'grey'}, 
    1: {color: 'brown'}, 
    2: {color: 'blue'}, 
    3: {color: 'green'},
}};

// Default band names don't sort propertly
// Instead, we can give a dictionary with
// labels for each band in the X-Axis
var bandDescriptions = {
  'B2': 'B02/Blue',
  'B3': 'B03/Green',
  'B4': 'B04/Red',
  'B8': 'B08/NIR',
  'B11': 'B11/SWIR-1',
  'B12': 'B12/SWIR-2'
}
// Create the chart and set options.
var chart = ui.Chart.feature.byProperty({
  features: fc,
  xProperties: bandDescriptions,
  seriesProperty: 'class'
})
.setChartType('ScatterChart')
.setOptions(options);

print(chart)

var classChart = function(landcover, label, color) {
  var options = {
  title: 'Spectral Signatures for ' + label + ' Class',
  hAxis: {title: 'Bands'},
  vAxis: {title: 'Reflectance'},
  lineWidth: 1,
  pointSize: 4,
  };

  var fc = training.filter(ee.Filter.eq('landcover', landcover))
  var chart = ui.Chart.feature.byProperty({
  features: fc,
  xProperties: bandDescriptions,
  })
.setChartType('ScatterChart')
.setOptions(options);

print(chart)
}
classChart(0, 'Urban')
classChart(1, 'Bare')
classChart(2, 'Water')
classChart(3, 'Vegetation')
"""
geemap.js_snippet_to_py(js_snippet,add_new_cell=True,show_map=True)

In [32]:

gcps = ee.FeatureCollection("LANDSAT/LC08/C02/T1_L2")\
    .filterBounds(ee_kassanje_polygon_feature) \
        .filterDate('2020-01-01', '2024-08-01') \
            .filterMetadata('CLOUD_COVER', 'less_than', 10)
composite = ee.Image('LANDSAT/LC08/C02/T1_L2_composite')

# Overlay the point on the image to get bands data.
training = composite.sampleRegions(
collection=gcps,
properties=['landcover'],
scale=30
)

# We will create a chart of spectral signature for all classes

# We have multiple GCPs for each class
# Use a grouped reducer to calculate the average reflectance
# for each band for each class

# We have 12 bands so need to repeat the reducer 12 times
# We also need to group the results by class
# So we find the index of the landcover property and use it
# to group the results
bands = composite.bandNames()
numBands = bands.length()
bandsWithClass = bands.add('landcover')
classIndex = bandsWithClass.indexOf('landcover')

# Use .combine() to get a reducer capable of
# computing multiple stats on the input
combinedReducer = ee.Reducer.mean().combine(
reducer2=ee.Reducer.stdDev(),
sharedInputs=True)

# Use .repeat() to get a reducer for each band
# We then use .group() to get stats by class
repeatedReducer = combinedReducer.repeat(numBands).group(classIndex)

gcpStats = training.reduceColumns(
selectors=bands.add('landcover'),
reducer=repeatedReducer,
)

# Result is a dictionary, we do some post-processing to
# extract the results
groups = ee.List(gcpStats.get('groups'))

classNames = ee.List(['urban', 'bare', 'water', 'vegetation'])

def func_faz(item) =
    # Extract the means
    values = ee.Dictionary(item).get('mean')
    groupNumber = ee.Dictionary(item).get('group')
    properties = ee.Dictionary.fromLists(bands, values)
    withClass = properties.set('class', classNames.get(groupNumber))
    return ee.Feature(None, withClass)

fc = ee.FeatureCollection(groups.map(func_faz

))

# Chart spectral signatures of training data
options = {{
    'title': 'Average Spectral Signatures',
    'hAxis': '{title': 'Bands'},
    vAxis={title='Reflectance'},
    lineWidth=1,
    pointSize=4,
    series={
                0={color='grey'},
                1={color='brown'},
                2={color='blue'},
                3={color='green'},
}}

# Default band names don't sort propertly
# Instead, we can give a dictionary with
# labels for each band in the X-Axis
bandDescriptions = {
        'B2': 'B02/Blue',
        'B3': 'B03/Green',
        'B4': 'B04/Red',
        'B8': 'B08/NIR',
        'B11': 'B11/SWIR-1',
        'B12': 'B12/SWIR-2'
}
# Create the chart and set options.
chart = ui.Chart.feature.byProperty(
features=fc,
xProperties=bandDescriptions,
seriesProperty='class'
) \
.setChartType('ScatterChart') \
.setOptions(options)

print(chart.getInfo())

def func_wfa(landcover, label, color) =
    options = {
                'title': 'Spectral Signatures for ' + label + ' Class',
                'hAxis': '{title': 'Bands'},
                vAxis={title='Reflectance'},
                lineWidth=1,
                pointSize=4 }

    fc = training.filter(ee.Filter.eq('landcover', landcover))
    chart = ui.Chart.feature.byProperty(
    features=fc,
    xProperties=bandDescriptions,
        ) \
    .setChartType('ScatterChart') \
    .setOptions(options)

    print(chart.getInfo())

classChart = func_wfa

classChart(0, 'Urban')
classChart(1, 'Bare')
classChart(2, 'Water')
classChart(3, 'Vegetation')
Map

SyntaxError: invalid syntax (207600758.py, line 50)